In [1]:
%matplotlib inline

# Housing Inventory - Get

This notebook creates the SQLite database and imports the data from the staged data sources bundled in the package.

## 1. Database Construction

The database can be constructed by running the DDL file as stdin to the database file as follows:

```
sqlite3 housing_inventory.db < create_db.sql

```
Alternatively, it can be created by running this cell:

In [2]:
import os
os.system('sqlite3 housing_inventory.db < create_db.sql')

0

This creates the various tables, populates the staging information, and then aggregates the data in a denormalized central table to facilitate analysis. This structure is illustrated below as:  ![db table layout](images/db%20table%20layout.jpg)


## 2. Pre-Processing
- Housing Inventory
- Number of Building Permits
- Mortgage Rates
- Prime Interest Rates
- Millions of Dollars in Revolving Credit


## 3. Import Process

First, let's stage some helper functions we will reuse.

In [3]:
import openpyxl as op
import pandas as pd
import sqlite3
from datetime import datetime

def create_connection(db_file: str) -> dict:
    """
    Create a database connection to the SQLite database specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    db={}
    db['con'] = None
    db['cur'] = None
    try:
        db['con'] = sqlite3.connect(db_file)
        db['cur'] = db['con'].cursor()
    except sqlite3.Error as e:
        print(e)
    return db


def close_connection(db_conn: sqlite3.Connection) -> None:
    return db_conn.close()


db_file = 'housing_inventory.db'

### 3.1 Import Housing Inventory

In [4]:
src_csv = 'src data/RDC_Inventory_Core_Metrics_Metro_History.csv'
main_columns = ['month_date_yyyymm', 'cbsa_code', 'cbsa_title', 'total_listing_count']

def get_info(t_file: str) -> pd.DataFrame:
    """
    Get the info out of the csv file and handle the year month breakout
    :param t_file:
    :return:
    """
    df = pd.read_csv(t_file, header=0)
    df = df.loc[:, df.columns.isin(main_columns)]
    return df


def fill_db(t_df: pd.DataFrame, db: dict) -> None:
    """
    Take the dictionary and fill the database
    """
    con = db['con']
    cur = db['cur']

    entries = []
    for index, row in t_df.iterrows():
        entries.append([row.month_date_yyyymm, row.cbsa_code, row.cbsa_title, row.total_listing_count])
    cur.executemany(
        'INSERT into housing_inventory (year_month, cbsa_code, cbsa_title, total_listing_count) VALUES (?, ?, ?, ?)',
        entries)
    con.commit()
    return

def main(t_db_file=db_file):
    housing_inventory_df = get_info(src_csv)
    db_ref = create_connection(t_db_file)
    fill_db(housing_inventory_df, db_ref)
    close_connection(db_ref['con'])
    return

main()

### 3.2 Import Building Permits

Importing the building permits is a bit more involved. The data from 2016-2018 was in tabular text data while the 2019 and 2020 data was in Excel format. In the spirit of normalizing the import int the database, the textual files were manually converted to Microsoft Excel format.

In [5]:
master_tab = 'MSA Units'

excel_files = ['src data/building permits/2019 building permits.xlsx',
               'src data/building permits/2020 building permits.xlsx',
               'src data/building permits/2016 units.xlsx',
               'src data/building permits/2017 units.xlsx',
               'src data/building permits/2018 units.xlsx']
def get_padded_months() -> list:
    padded_months = [''.join(['0', str(x)]) for x in range(1, 10)]
    padded_months.append('11')
    padded_months.append('12')
    return padded_months


def get_spreadsheet_info(t_file: str, t_padded_months: list) -> list:
    """
    Get the info out of the csv file and handle the year month breakout
    :param t_file:
    :return:
    """
    if '2020' in t_file:
        year = '2020'
    elif '2019' in t_file:
        year = '2019'
    elif '2018' in t_file:
        year = '2017'
    elif '2017' in t_file:
        year = '2017'
    elif '2016' in t_file:
        year = '2016'

    wb = op.load_workbook(t_file)
    results = []
    master_sheet = wb[master_tab]
    for cell_value in master_sheet.iter_rows(min_row=8, min_col=1, max_col=4, values_only=True):
        if cell_value[0] is None: break
        cbsa_code = cell_value[1]
        monthly_est = int(cell_value[3]) / 12  # we have annual rollups, let's estimate via mean
        month_entries = [[''.join([year, x]), cbsa_code, monthly_est] for x in t_padded_months]
        [results.append(month) for month in month_entries]
    return results


def fill_db(t_entries: list, db: dict) -> None:
    """
    Take the dictionary and fill the database
    """
    con = db['con']
    cur = db['cur']
    cur.executemany(
        'INSERT into building_permits(year_month, cbsa_code, total_units) VALUES (?, ?, ?)',
        t_entries)
    con.commit()
    return


def main(t_db_file = db_file):
    padded_months = get_padded_months()
    db_ref = create_connection(t_db_file)
    for spreadsheet in excel_files:
         permits = get_spreadsheet_info(spreadsheet, padded_months)
         fill_db(permits, db_ref)
    close_connection(db_ref['con'])
    return


main()



### 3.3 Import Mortgage Rates

In [6]:
master_tab = 'Full History'
src_file = 'src data/historical weekly mortgage data.xlsx'

def get_info(t_file: str) -> list:
    """
    Get the info out of the csv file and handle the year month breakout
    :param t_file:
    :return:
    """
    wb = op.load_workbook(t_file)
    results=[]
    master_sheet = wb[master_tab]
    for cell_value in master_sheet.iter_rows(min_row=8, min_col=0, max_col=2, values_only=True):
        if cell_value[0] is None: break
        t_date = cell_value[0]
        new_date = datetime.strftime(t_date,'%Y%m')
        # only retain the records after June 2016 for import into the db
        if new_date > '201606':
            results.append([new_date, cell_value[1]])
    return results


def fill_db(t_entries: list, db: dict) -> None:
    """
    Take the dictionary and fill the database
    """
    con = db['con']
    cur = db['cur']

    cur.executemany(
        'INSERT into mortgage_rates (year_month, mortgage_rate) VALUES (?, ?)',
        t_entries)
    con.commit()
    return


def main(t_db_file = db_file):
    mortgage_inventory = get_info(src_file)
    db_ref = create_connection(t_db_file)
    fill_db(mortgage_inventory, db_ref)
    close_connection(db_ref['con'])
    return

main()

### 3.5 Import Prime Rate

In [7]:
master_tab = 'Sheet1'
src_file = 'src data/Prime Interest Rates.xlsx'


def get_info(t_file: str) -> list:
    """
    Get the info out of the csv file and handle the year month breakout
    :param t_file:
    :return:
    """
    wb = op.load_workbook(t_file)
    results=[]
    master_sheet = wb[master_tab]
    for cell_value in master_sheet.iter_rows(min_row=7, min_col=0, max_col=2, values_only=True):
        if cell_value[0] is None: break
        new_date = datetime.strptime(cell_value[0],'%Y-%m')
        new_date = datetime.strftime(new_date, '%Y%m')
        # only retain the records after June 2016 for import into the db
        if new_date > '201606':
            results.append([new_date, cell_value[1]])
    return results


def fill_db(t_entries: list, db: dict) -> None:
    """
    Take the dictionary and fill the database
    """
    con = db['con']
    cur = db['cur']
    cur.executemany(
        'INSERT into prime_rates(year_month, prime_rate) VALUES (?, ?)',
        t_entries)
    con.commit()
    return


def main(t_db_file = db_file):
    prime_rates= get_info(src_file)
    db_ref = create_connection(t_db_file)
    fill_db(prime_rates, db_ref)
    close_connection(db_ref['con'])
    return


main()

### 3.5 Import Revolving Credit

In [8]:
master_tab = 'FRB_G19'
src_file = 'src data/Revolving Credit.xlsx'


def get_info(t_file: str) -> list:
    """
    Get the info out of the csv file and handle the year month breakout
    :param t_file:
    :return:
    """
    wb = op.load_workbook(t_file)
    results=[]
    master_sheet = wb[master_tab]
    for cell_value in master_sheet.iter_rows(min_row=7, min_col=0, max_col=2, values_only=True):
        if cell_value[0] is None: break
        new_date = datetime.strptime(cell_value[0],'%Y-%m')
        new_date = datetime.strftime(new_date, '%Y%m')
        # only retain the records after June 2016 for import into the db
        if new_date > '201606':
            results.append([new_date, cell_value[1]])
    return results


def fill_db(t_entries: list, db: dict) -> None:
    """
    Take the dictionary and fill the database
    """
    con = db['con']
    cur = db['cur']
    cur.executemany(
        'INSERT into revolving_credit(year_month, credit) VALUES (?, ?)',
        t_entries)
    con.commit()
    return


def main(t_db_file = db_file):
    credit = get_info(src_file)
    db_ref = create_connection(t_db_file)
    fill_db(credit, db_ref)
    close_connection(db_ref['con'])
    return

main()